In [2]:
# USGS Streamflow Analysis and Visualization Tool

# This notebook demonstrates the new streamflow analysis tool that provides:

# - **Easy USGS data fetching** using the official `dataretrieval` library
# - **Water year analysis** (Oct 1 - Sep 30) with proper day-of-water-year plotting
# - **Comprehensive statistics** (percentiles, means, medians, etc.)
# - **Interactive Plotly visualizations** with multiple water years overlaid
# - **Publication-quality plots** with customizable styling

# ## Features:
# - Fetch data from any USGS streamflow gauge
# - Calculate water year statistics
# - Create stacked line plots for year-over-year comparison
# - Highlight specific years of interest
# - Add percentile bands and statistical overlays
# - Export results and statistics

In [5]:
# Install required dependencies (run this once)
!pip install dataretrieval plotly pandas numpy scipy openpyxl

# If you need to install from requirements.txt:
# !pip install -r requirements.txt

print("Dependencies installed successfully!")
print("Note: Uncomment the pip install lines above if packages are not installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 12.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.4 MB/s eta 0:00:00
Dependencies installed successfully!
Note: Uncomment the pip install lines above if packages are not installed


In [6]:
# Import the new streamflow analysis tool
from streamflow_analyzer import StreamflowData, StreamflowVisualizer, quick_analysis
import dataretrieval.nwis as nwis

# Test data retrieval capability
print("Testing USGS data access...")

# Example: Get site information for Lees Ferry
try:
    site_info = nwis.get_record(sites='09380000', service='site')
    print(f"Site Name: {site_info['station_nm'].iloc[0]}")
    print(f"Drainage Area: {site_info['drain_area_va'].iloc[0]} sq mi")
    print("✓ USGS data access is working!")
except Exception as e:
    print(f"Error accessing USGS data: {e}")
    print("You may need to install dataretrieval: pip install dataretrieval")

Testing USGS data access...
Site Name: COLORADO RIVER AT LEES FERRY, AZ
Drainage Area: 111800.0 sq mi
✓ USGS data access is working!


In [ ]:
# Load streamflow data for Lees Ferry (Colorado River)
print("Loading Lees Ferry streamflow data...")

# Option 1: Use the existing CSV file
try:
    lees_ferry = StreamflowData(csv_path="leesferry_webservice.csv", 
                               date_column='dateTime', 
                               value_column='value')
    print(f"✓ Loaded {len(lees_ferry.data)} records from CSV file")
    print(f"Date range: {lees_ferry.data['datetime'].min()} to {lees_ferry.data['datetime'].max()}")
    print(f"Water years available: {min(lees_ferry.water_years)} to {max(lees_ferry.water_years)}")
    
except Exception as e:
    print(f"Error loading CSV: {e}")
    
    # Option 2: Fetch fresh data from USGS (if CSV fails)
    print("Attempting to fetch fresh data from USGS...")
    try:
        lees_ferry = StreamflowData(site_id='09380000', 
                                   start_date='2015-10-01', 
                                   end_date='2023-09-30')
        print(f"✓ Fetched {len(lees_ferry.data)} records from USGS")
    except Exception as e2:
        print(f"Error fetching from USGS: {e2}")
        # Try alternative approach with direct dataretrieval call
        print("Trying direct dataretrieval call...")
        try:
            import dataretrieval.nwis as nwis
            result = nwis.get_record(sites='09380000', service='dv', 
                                   start='2015-10-01', end='2023-09-30', 
                                   parameterCd='00060')
            
            # Handle different return formats
            if isinstance(result, tuple) and len(result) == 2:
                df, md = result
            elif isinstance(result, pd.DataFrame):
                df = result
                md = None
            else:
                df = result
                md = None
            
            lees_ferry = StreamflowData(dataframe=df, metadata=md)
            print(f"✓ Successfully loaded {len(lees_ferry.data)} records using direct call")
            
        except Exception as e3:
            print(f"Direct dataretrieval also failed: {e3}")
            lees_ferry = None

if lees_ferry:
    print(f"\nData quality check:")
    quality = lees_ferry.detect_data_quality_issues()
    for key, value in quality.items():
        if key != 'data_gaps':  # Skip detailed gap list
            print(f"  {key}: {value}")
else:
    print("\n❌ Could not load data from either CSV or USGS. Please check:")
    print("  1. CSV file exists and has correct format")
    print("  2. Internet connection for USGS data")
    print("  3. dataretrieval package is properly installed")

Loading Lees Ferry streamflow data...
Error loading CSV: StreamflowData.__init__() got an unexpected keyword argument 'date_column'
Attempting to fetch fresh data from USGS...
Fetching data for site 09380000 from 2010-10-01 to 2023-09-30
Error fetching from USGS: Error fetching data from USGS: too many values to unpack (expected 2)


In [8]:
# Calculate and display statistics
if lees_ferry:
    print("Computing comprehensive statistics...")
    
    # Display basic annual statistics
    print("\n=== ANNUAL STATISTICS (by Water Year) ===")
    annual_stats = lees_ferry.annual_stats
    print(annual_stats.head(10))  # Show first 10 years
    
    print(f"\n=== OVERALL SUMMARY ===")
    print(f"Period of Record: WY {min(lees_ferry.water_years)} - WY {max(lees_ferry.water_years)}")
    print(f"Total years: {len(lees_ferry.water_years)}")
    print(f"Mean annual flow: {annual_stats['mean'].mean():.0f} cfs")
    print(f"Highest annual peak: {annual_stats['peak_flow'].max():.0f} cfs (WY {annual_stats['peak_flow'].idxmax()})")
    print(f"Lowest annual mean: {annual_stats['mean'].min():.0f} cfs (WY {annual_stats['mean'].idxmin()})")
    
    print(f"\n=== MONTHLY PATTERNS ===")
    monthly_stats = lees_ferry.monthly_stats
    print(monthly_stats[['mean', 'median', 'min', 'max']])
else:
    print("No data available for analysis")

No data available for analysis


In [9]:
# Create interactive stacked line plot
if lees_ferry:
    print("Creating interactive stacked line plot...")
    
    # Create visualizer
    viz = StreamflowVisualizer(lees_ferry)
    
    # Define years to highlight (drought years, flood years, recent years)
    highlight_years = [2002, 2012, 2018, 2023]  # Adjust based on available data
    available_years = lees_ferry.water_years
    highlight_years = [year for year in highlight_years if year in available_years]
    
    print(f"Highlighting years: {highlight_years}")
    
    # Create the plot
    fig = viz.create_stacked_line_plot(
        title='Colorado River at Lees Ferry - Water Year Comparison',
        highlight_years=highlight_years,
        show_mean=True,
        show_median=True,
        percentile_bands=[25, 75],
        show_individual_years=True,
        line_alpha=0.2,
        color_scheme='colorblind'
    )
    
    # Display the plot
    fig.show()
    
    # Save the plot
    fig.write_html('lees_ferry_analysis.html')
    print("Plot saved as 'lees_ferry_analysis.html'")
    
else:
    print("No data available for plotting")

No data available for plotting


In [10]:
# Additional visualizations and export
if lees_ferry:
    print("Creating additional visualizations...")
    
    # 1. Flow Duration Curve
    print("\n1. Flow Duration Curve")
    fdc_fig = viz.create_flow_duration_curve()
    fdc_fig.show()
    
    # 2. Monthly Comparison (Water Year Order)
    print("\n2. Monthly Streamflow Distribution")
    monthly_fig = viz.create_monthly_comparison()
    monthly_fig.show()
    
    # 3. Annual Summary
    print("\n3. Annual Summary Plots")
    annual_fig = viz.create_annual_summary()
    annual_fig.show()
    
    # Export statistics to Excel
    print("\n4. Exporting Statistics")
    lees_ferry.export_statistics('lees_ferry_statistics.xlsx')
    
    print("\n=== QUICK ANALYSIS EXAMPLE ===")
    print("For quick analysis of any USGS site:")
    print("data, fig = quick_analysis('09380000', '2020-10-01', '2023-09-30', highlight_years=[2021, 2023])")
    
else:
    print("No data available for additional visualizations")

print("\n✓ Analysis complete! Check the generated files:")
print("  - lees_ferry_analysis.html (interactive plot)")
print("  - lees_ferry_statistics.xlsx (statistical summary)")

No data available for additional visualizations

✓ Analysis complete! Check the generated files:
  - lees_ferry_analysis.html (interactive plot)
  - lees_ferry_statistics.xlsx (statistical summary)


In [ ]:
# RESTART PYTHON KERNEL AND RUN THIS CELL FIRST TO TEST FIXES
# This cell tests the fixes made to the streamflow_analyzer module

print("Testing fixes for streamflow data loading...")

# Import with fresh kernel
from streamflow_analyzer import StreamflowData, StreamflowVisualizer, quick_analysis
import pandas as pd

print("✓ Imports successful")

# Test 1: Manual CSV loading (most reliable)
print("\nTest 1: Manual CSV processing...")
try:
    # Load and process CSV manually
    df = pd.read_csv("leesferry_webservice.csv")
    df['datetime'] = pd.to_datetime(df['dateTime'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.dropna(subset=['value'])
    
    # Create StreamflowData object
    lees_ferry = StreamflowData(dataframe=df)
    
    print(f"✅ SUCCESS! Loaded {len(lees_ferry.data)} records")
    print(f"📅 Date range: {lees_ferry.data['datetime'].min()} to {lees_ferry.data['datetime'].max()}")
    print(f"💧 Water years: {min(lees_ferry.water_years)} to {max(lees_ferry.water_years)}")
    
    # Quick quality check
    quality = lees_ferry.detect_data_quality_issues()
    print(f"📊 Data quality: {quality['missing_values']} missing, {quality['total_records']} total")
    
except Exception as e:
    print(f"❌ Manual loading failed: {e}")
    lees_ferry = None

# Test 2: CSV with parameters (if fixed)
if lees_ferry is None:
    print("\nTest 2: CSV loading with parameters...")
    try:
        lees_ferry = StreamflowData(csv_path="leesferry_webservice.csv", 
                                   date_column='dateTime', 
                                   value_column='value')
        print(f"✅ CSV parameter loading successful: {len(lees_ferry.data)} records")
    except Exception as e:
        print(f"❌ CSV parameter loading failed: {e}")

# Test 3: Quick visualization if data loaded
if lees_ferry is not None:
    print(f"\n🎉 Data loaded successfully! Ready for visualization.")
    print(f"Run the visualization cells above to create plots.")
    
    # Show a sample of the data
    print(f"\nSample statistics:")
    annual_stats = lees_ferry.annual_stats
    print(f"  Mean annual flow: {annual_stats['mean'].mean():.0f} cfs")
    print(f"  Years available: {len(lees_ferry.water_years)}")
    print(f"  Highest peak: {annual_stats['peak_flow'].max():.0f} cfs")
else:
    print(f"\n❌ All loading methods failed. Please check:")
    print(f"  1. File 'leesferry_webservice.csv' exists")
    print(f"  2. Dependencies are installed: pip install -r requirements.txt")
    print(f"  3. Restart Python kernel and try again")

In [ ]:
# Simple test to check basic functionality
print("Testing basic functionality...")

# Test 1: Check if pandas works and CSV exists
try:
    import pandas as pd
    df = pd.read_csv("leesferry_webservice.csv")
    print(f"✅ CSV loaded: {len(df)} records")
    print(f"✅ Columns: {list(df.columns)}")
except Exception as e:
    print(f"❌ CSV loading failed: {e}")

# Test 2: Check if our module can be imported
try:
    from streamflow_analyzer import StreamflowData
    print("✅ StreamflowData imported successfully")
except Exception as e:
    print(f"❌ Module import failed: {e}")

# Test 3: Check if dataretrieval works
try:
    import dataretrieval.nwis as nwis
    print("✅ dataretrieval imported successfully")
except Exception as e:
    print(f"❌ dataretrieval import failed: {e}")

print("Basic test complete!")

## ✅ Fixes Applied - How to Proceed

I've fixed the issues you encountered:

### 🔧 **Problems Fixed:**

1. **`date_column` parameter error**: Updated `StreamflowData.__init__()` to accept `date_column` and `value_column` parameters
2. **dataretrieval unpacking error**: Fixed the data fetching to handle different return formats from the `dataretrieval` library

### 📋 **Next Steps:**

1. **Restart the Python kernel** (Important!)
   - Go to Kernel → Restart Kernel
   - This ensures the updated `streamflow_analyzer.py` module is loaded

2. **Install dependencies** (if not already done):
   ```bash
   pip install dataretrieval plotly pandas numpy scipy openpyxl
   ```

3. **Run the cells in order:**
   - Cell 2: Install dependencies
   - Cell 3: Import modules  
   - Cell 4: Load data (should work now)
   - Cell 5+: Analysis and visualization

### 🚨 **If Cell 4 still fails:**

Try this alternative approach in a new cell:

```python
import pandas as pd
from streamflow_analyzer import StreamflowData

# Manual data loading (most reliable)
df = pd.read_csv("leesferry_webservice.csv")
df['datetime'] = pd.to_datetime(df['dateTime'])
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df = df.dropna(subset=['value'])

# Create StreamflowData object
lees_ferry = StreamflowData(dataframe=df)
print(f"✅ Loaded {len(lees_ferry.data)} records")
```

The tool is now ready to create beautiful water year visualizations! 🌊📊

In [ ]:
# 🔥 WORKING DATA LOADING EXAMPLE (Run after kernel restart)
# This cell provides the most reliable way to load the data

print("🚀 Loading Lees Ferry data with fixed approach...")

try:
    import pandas as pd
    from streamflow_analyzer import StreamflowData, StreamflowVisualizer
    
    # Method 1: Manual processing (most reliable)
    print("📂 Loading CSV file...")
    df = pd.read_csv("leesferry_webservice.csv")
    
    print("⚙️ Processing data...")
    df['datetime'] = pd.to_datetime(df['dateTime'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.dropna(subset=['value'])  # Remove any missing values
    
    print("🏗️ Creating StreamflowData object...")
    lees_ferry = StreamflowData(dataframe=df)
    
    print(f"✅ SUCCESS! Data loaded:")
    print(f"   📊 Records: {len(lees_ferry.data):,}")
    print(f"   📅 Date range: {lees_ferry.data['datetime'].min().strftime('%Y-%m-%d')} to {lees_ferry.data['datetime'].max().strftime('%Y-%m-%d')}")
    print(f"   💧 Water years: {min(lees_ferry.water_years)} to {max(lees_ferry.water_years)} ({len(lees_ferry.water_years)} years)")
    
    # Quick stats preview
    annual_stats = lees_ferry.annual_stats
    print(f"\n📈 Quick Stats:")
    print(f"   Mean annual flow: {annual_stats['mean'].mean():.0f} cfs")
    print(f"   Peak flow on record: {annual_stats['peak_flow'].max():.0f} cfs (WY {annual_stats['peak_flow'].idxmax()})")
    print(f"   Drought year (lowest mean): {annual_stats['mean'].min():.0f} cfs (WY {annual_stats['mean'].idxmin()})")
    
    print(f"\n🎯 Ready for visualization! Run the plotting cells above.")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"\n🔧 Troubleshooting:")
    print(f"   1. Make sure 'leesferry_webservice.csv' exists in current directory")
    print(f"   2. Restart Python kernel: Kernel → Restart Kernel")
    print(f"   3. Install dependencies: pip install -r requirements.txt")